In [1]:
import pandas as pd
import re
from gensim.models import Word2Vec
import gensim
def cleanData(csvLocation, name="default"):
    
    def clean140(data):
        data = pd.read_csv(data, encoding = "latin1")
        data.columns=['sentiment', 'id', 'date', 'q', 'user', 'tweet']
        data = data.drop(['id', 'date', 'q', 'user'], axis=1)
        data = shuffle(data)
        text = data.tweet
        sentiment = data.sentiment
        sentiment = pd.Series.tolist(sentiment)
        for i in range(len(sentiment)):
            j = sentiment[i]
            if j==4:
                sentiment[i]=1
            elif j==2:
                sentiment[i]=0.5

        sentiment=np.asarray(sentiment)

        trainData = []
        for x in text:
            trainData.append(''.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",x.lower())).split())
        
        return (trainData, sentiment)



    if name == "140":
        data, sentiment = clean140(csvLocation)
        return (data, sentiment)
        

    data = pd.read_csv(csvLocation, sep='\t')
    data.columns = ['text', 'sentiment']
    trainData=[]
    for x in data.text:
        trainData.append(''.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",x.lower())).split())
        
    return (trainData, data.sentiment)

In [14]:
max_features = 5000
maxlen = 50
batch_size = 50
embedding_dims = 50
filters = 250
kernel_size = 3
hidden_dims = 250
epochs = 2

In [3]:
import gensim
import os.path
import tensorflow as tf
import pandas as pd
from sklearn.utils import shuffle
import re
import numpy as np
from gensim.models import Word2Vec
from keras.layers import Conv1D, GlobalAveragePooling1D, MaxPooling1D
from keras.layers.pooling import GlobalMaxPooling1D
from keras.utils import to_categorical
import json

/home/abd/testEnv/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [4]:
sentence, sentiment  = cleanData("sentiment140train.csv", name= "140")

In [5]:
if os.path.exists('trainedW2V'):
    wordModel=gensim.models.Word2Vec.load("trainedW2V")
else:
    wordModel=gensim.models.Word2Vec(sentence, min_count=1, size=300, max_vocab_size=50000, iter=50, workers=50)
    wordModel.save('trainedW2V')


In [6]:
pretrained_weights = wordModel.wv.syn0
vocab_size, embedding_size = pretrained_weights.shape

def word2idx(word):
    return wordModel.wv.vocab[word].index
def idx2word(idx):
    return wordModel.wv.index2word[idx]

/home/abd/testEnv/lib/python3.5/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
  """Entry point for launching an IPython kernel.


In [7]:
pretrained_weights = wordModel.wv.syn0
vocab_size, emdedding_size = pretrained_weights.shape

/home/abd/testEnv/lib/python3.5/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
  """Entry point for launching an IPython kernel.


In [8]:
trainData = np.zeros([len(sentence), maxlen], dtype=np.int32)
for i, sentences in enumerate(sentence):
  for t, word in enumerate(sentences[:-1]):
    try:
        trainData[i, t] = word2idx(word)
    except:
        print(word + " not found!")
        pass
            

print(trainData.shape)

the not found!
other not found!
th not found!
(1599999, 50)


In [21]:
from keras.models import Sequential
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers.embeddings import Embedding
from keras.layers.core import Dense, Dropout, Flatten, Activation
from keras.layers.convolutional import Convolution1D, MaxPooling1D
from keras.optimizers import Adamax

In [22]:
with tf.device('/gpu:2'):
    print('Build model...')
    model = Sequential()

    # we start off with an efficient embedding layer which maps
    # our vocab indices into embedding_dims dimensions
    model.add(Embedding(max_features,
                        embedding_dims,
                        input_length=maxlen))

    model.add(Dropout(0.5))

    # we add a Convolution1D, which will learn filters
    # word group filters of size filter_length:
    model.add(Conv1D(filters,
                     kernel_size,
                     padding='valid',
                     activation='relu',
                     strides=1))
    # we use max pooling:
    model.add(GlobalMaxPooling1D())

    # We add a vanilla hidden layer:
    model.add(Dense(hidden_dims))
    model.add(Dropout(0.2))
    model.add(Activation('relu'))

    # We project onto a single unit output layer, and squash it with a sigmoid:
    model.add(Dense(1))
    model.add(Activation('sigmoid'))

    model.compile(loss='binary_crossentropy',
                  optimizer=Adamax(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0),
                  metrics=['accuracy'])
    model.summary()

Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 50, 50)            250000    
_________________________________________________________________
dropout_13 (Dropout)         (None, 50, 50)            0         
_________________________________________________________________
conv1d_7 (Conv1D)            (None, 48, 250)           37750     
_________________________________________________________________
global_max_pooling1d_7 (Glob (None, 250)               0         
_________________________________________________________________
dense_13 (Dense)             (None, 250)               62750     
_________________________________________________________________
dropout_14 (Dropout)         (None, 250)               0         
_________________________________________________________________
activation_13 (Activation)   (None, 250)               0     

In [23]:
model.fit(trainData, sentiment, validation_split=0.20, epochs=10, batch_size=50)

Train on 1279999 samples, validate on 320000 samples
Epoch 1/10
1279999/1279999 [==============================] - 147s 115us/step - loss: 0.5480 - acc: 0.7221 - val_loss: 0.5419 - val_acc: 0.7110
Epoch 2/10
1279999/1279999 [==============================] - 145s 113us/step - loss: 0.5344 - acc: 0.7223 - val_loss: 0.5255 - val_acc: 0.7289
Epoch 3/10
1279999/1279999 [==============================] - 145s 113us/step - loss: 0.5199 - acc: 0.7384 - val_loss: 0.5099 - val_acc: 0.7457
Epoch 4/10
1279999/1279999 [==============================] - 145s 113us/step - loss: 0.5089 - acc: 0.7478 - val_loss: 0.4977 - val_acc: 0.7549
Epoch 5/10
1279999/1279999 [==============================] - 146s 114us/step - loss: 0.5011 - acc: 0.7550 - val_loss: 0.4879 - val_acc: 0.7625
Epoch 6/10
1279999/1279999 [==============================] - 145s 113us/step - loss: 0.4960 - acc: 0.7584 - val_loss: 0.4845 - val_acc: 0.7653
Epoch 7/10
1279999/1279999 [==============================] - 145s 113us/step - los